In [36]:
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# Importing Gensim for LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# PLotting tools 

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('C:/Users/abhis/OneDrive/Documents/GitHub/PeopleTail_NLP/src/reviews.csv')
df.count(), df.columns


(body        1873
 watson      1873
 products    1824
 user        1873
 dtype: int64, Index(['body', 'watson', 'products', 'user'], dtype='object'))

In [3]:
df['reviewtext'] = df['body'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

In [4]:
df.head()

,body,watson,products,user,reviewtext
0,"<p class=""MsoNormal"">Ah spring time is on the ...","{""usage"":{""text_units"":1,""text_characters"":230...",True Moringa - All Purpose Moringa Body Oil wi...,5,Ah spring time is on the horizon. Why not give...
1,"<p dir=""ltr"" style=""line-height: 1.38; margin-...","{""usage"":{""text_units"":1,""text_characters"":280...",BioFinest Pomegranate Seed Organic Oil - 100% ...,5,Feeling like you need a kick in the pants to g...
2,"<p class=""MsoNormal"">Oral health is key for ov...","{""usage"":{""text_units"":1,""text_characters"":133...","Lakanto Monkfruit Sweetener, Golden, 8.29 Ounc...",5,Oral health is key for overall health. Did yo...
3,"<p class=""MsoNormal"">The benefits of adding le...","{""usage"":{""text_units"":1,""text_characters"":117...",NaN,5,The benefits of adding lemon to our drinking w...
4,"<p class=""MsoNormal"">Our diets today, unfortun...","{""usage"":{""text_units"":1,""text_characters"":210...",Gelatinized Maca Root Powder From Peru - Certi...,5,"Our diets today, unfortunately have many hidde..."


# Preprocess the data and normalize it

In [22]:
stop_words = set(stopwords.words('english'))

# Convert to list
data = df.reviewtext.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['Ah spring time is on the horizon. Why not give your mane a spring detox and make over? Dry and cold winds can deplete your hair and scalp of moisture. Overtime your locks can become dry and brittle. Try this easy and pampering recommendation to protect your mane to look lustrous and feel silky smooth again. Apply True Moringa - All Purpose Moringa Body Oil with Peppermint, to your hair and scalp, leave in hair for 30 minutes or wrap your hair in a towel and leave in overnight. Moringa oil does not clog scalp pores, strengthens hair follicles and combats frizz. Keep it in your hair overnight into the next day (really its not greasy at all?). Wash it out with a mild, sulphate free shampoo, like the very juicy ACURE Mega Moisture Shampoo and reveal an uber moisturized hair and scalp. Next give your scalp the sustained moisture lock it deserves. We love the organic hair mask for its triple threat recovery from the dry blustery cold air that shocked our scalp and hair during the winter mo

<>:7: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \s
<ipython-input-22-6db0ed0c89cc>:7: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [23]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(data))

print(data_words[:1])


[['ah', 'spring', 'time', 'is', 'on', 'the', 'horizon', 'why', 'not', 'give', 'your', 'mane', 'spring', 'detox', 'and', 'make', 'over', 'dry', 'and', 'cold', 'winds', 'can', 'deplete', 'your', 'hair', 'and', 'scalp', 'of', 'moisture', 'overtime', 'your', 'locks', 'can', 'become', 'dry', 'and', 'brittle', 'try', 'this', 'easy', 'and', 'pampering', 'recommendation', 'to', 'protect', 'your', 'mane', 'to', 'look', 'lustrous', 'and', 'feel', 'silky', 'smooth', 'again', 'apply', 'true', 'moringa', 'all', 'purpose', 'moringa', 'body', 'oil', 'with', 'peppermint', 'to', 'your', 'hair', 'and', 'scalp', 'leave', 'in', 'hair', 'for', 'minutes', 'or', 'wrap', 'your', 'hair', 'in', 'towel', 'and', 'leave', 'in', 'overnight', 'moringa', 'oil', 'does', 'not', 'clog', 'scalp', 'pores', 'strengthens', 'hair', 'follicles', 'and', 'combats', 'frizz', 'keep', 'it', 'in', 'your', 'hair', 'overnight', 'into', 'the', 'next', 'day', 'really', 'its', 'not', 'greasy', 'at', 'all', 'wash', 'it', 'out', 'with', '

In [24]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [25]:
print(data_words_nostops[:1])

[['ah', 'spring', 'time', 'horizon', 'give', 'mane', 'spring', 'detox', 'make', 'dry', 'cold', 'winds', 'deplete', 'hair', 'scalp', 'moisture', 'overtime', 'locks', 'become', 'dry', 'brittle', 'try', 'easy', 'pampering', 'recommendation', 'protect', 'mane', 'look', 'lustrous', 'feel', 'silky', 'smooth', 'apply', 'true', 'moringa', 'purpose', 'moringa', 'body', 'oil', 'peppermint', 'hair', 'scalp', 'leave', 'hair', 'minutes', 'wrap', 'hair', 'towel', 'leave', 'overnight', 'moringa', 'oil', 'clog', 'scalp', 'pores', 'strengthens', 'hair', 'follicles', 'combats', 'frizz', 'keep', 'hair', 'overnight', 'next', 'day', 'really', 'greasy', 'wash', 'mild', 'sulphate', 'free', 'shampoo', 'like', 'juicy', 'acure', 'mega', 'moisture', 'shampoo', 'reveal', 'uber', 'moisturized', 'hair', 'scalp', 'next', 'give', 'scalp', 'sustained', 'moisture', 'lock', 'deserves', 'love', 'organic', 'hair', 'mask', 'triple', 'threat', 'recovery', 'dry', 'blustery', 'cold', 'air', 'shocked', 'scalp', 'hair', 'winter

In [31]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [38]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [39]:
print(data_lemmatized[:1])

[['spring', 'give', 'spring', 'detox', 'make', 'dry', 'cold', 'wind', 'deplete', 'hair', 'overtime', 'lock', 'become', 'dry', 'brittle', 'try', 'easy', 'pamper', 'recommendation', 'mane', 'look', 'lustrous', 'feel', 'apply', 'true', 'moringa', 'purpose', 'body', 'oil', 'peppermint', 'hair', 'scalp', 'leave', 'hair', 'minute', 'wrap', 'hair', 'towel', 'leave', 'overnight', 'pore', 'strengthen', 'hair', 'follicle', 'combat', 'frizz', 'keep', 'hair', 'overnight', 'next', 'day', 'really', 'greasy', 'wash', 'mild', 'shampoo', 'acure', 'shampoo', 'moisturize', 'hair', 'next', 'give', 'sustain', 'moisture', 'lock', 'deserve', 'love', 'organic', 'hair', 'mask', 'triple', 'threat', 'recovery', 'dry', 'cold', 'shock', 'hair', 'apply', 'time', 'week', 'hair', 'milk', 'rise', 'ingredient', 'rise', 'water', 'balance', 'moisture', 'level', 'seed', 'oil', 'repair', 'nourish', 'hair', 'gently', 'cleanse', 'away', 'excess', 'harmful', 'product', 'diyer', 'make', 'simple', 'hair', 'mask', 'food', 'ingre

# Sentiment Analysis

In [40]:
analyzer = SentimentIntensityAnalyzer()

In [41]:
sentiment = df['reviewtext'].apply(lambda x: analyzer.polarity_scores(x))

In [42]:
df = pd.concat([df,sentiment.apply(pd.Series)],1)

In [43]:
df.head()

,body,watson,products,user,reviewtext,reviewtext_token,text_lemmatized,reviewtext_new,neg,neu,pos,compound
0,"<p class=""MsoNormal"">Ah spring time is on the ...","{""usage"":{""text_units"":1,""text_characters"":230...",True Moringa - All Purpose Moringa Body Oil wi...,5,Ah spring time is on the horizon. Why not give...,"[ah, spring, time, horizon, why, give, mane, s...","[ah, spring, time, horizon, why, give, mane, s...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.042,0.846,0.112,0.9860
1,"<p dir=""ltr"" style=""line-height: 1.38; margin-...","{""usage"":{""text_units"":1,""text_characters"":280...",BioFinest Pomegranate Seed Organic Oil - 100% ...,5,Feeling like you need a kick in the pants to g...,"[feeling, like, need, kick, pants, get, stuper...","[feeling, like, need, kick, pant, get, stuper,...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.072,0.731,0.197,0.9974
2,"<p class=""MsoNormal"">Oral health is key for ov...","{""usage"":{""text_units"":1,""text_characters"":133...","Lakanto Monkfruit Sweetener, Golden, 8.29 Ounc...",5,Oral health is key for overall health. Did yo...,"[oral, health, key, overall, health, did, know...","[oral, health, key, overall, health, did, know...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.071,0.787,0.142,0.9473
3,"<p class=""MsoNormal"">The benefits of adding le...","{""usage"":{""text_units"":1,""text_characters"":117...",NaN,5,The benefits of adding lemon to our drinking w...,"[the, benefits, adding, lemon, drinking, water...","[the, benefit, adding, lemon, drinking, water,...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.015,0.762,0.224,0.9924
4,"<p class=""MsoNormal"">Our diets today, unfortun...","{""usage"":{""text_units"":1,""text_characters"":210...",Gelatinized Maca Root Powder From Peru - Certi...,5,"Our diets today, unfortunately have many hidde...","[our, diets, today, unfortunately, many, hidde...","[our, diet, today, unfortunately, many, hidden...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.034,0.827,0.138,0.9833


# LDA topic modelling

In [44]:
id2word = corpora.Dictionary(df['text_lemmatized'])

In [46]:
texts = data_lemmatized

In [47]:
corpus = [id2word.doc2bow(text) for text in texts]

In [54]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [55]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\abhis\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.183150  0.259338       1        1  36.857666
3      0.290486 -0.122811       2        1  29.398811
4      0.262810 -0.082634       3        1  24.994579
1     -0.185034 -0.016398       4        1   3.338497
2     -0.185503  0.001470       5        1   2.949466
0     -0.185274 -0.020240       6        1   1.283492
5     -0.180636 -0.018725       7        1   1.177498, topic_info=      Category         Freq          Term        Total  loglift  logprob
56     Default   379.000000          hair   379.000000  30.0000  30.0000
301    Default  1116.000000          skin  1116.000000  29.0000  29.0000
110    Default  1773.000000       product  1773.000000  28.0000  28.0000
174    Default   863.000000           add   863.000000  27.0000  27.0000
487    Default  2529.000000           use  2529.000000  26.0000  26.0000
3234   Default   269.000000      collagen   269.000000  25.0000  25.0000
72     Default  1098.000000          love  1098.000000  24.0000  24.0000
285    Default   589.000000       protein   589.000000  23.0000  23.0000
230    Default  1164.000000          help  1164.000000  22.0000  22.0000
421    Default   569.000000         taste   569.000000  21.0000  21.0000
158    Default   660.000000       vitamin   660.000000  20.0000  20.0000
96     Default   889.000000           oil   889.000000  19.0000  19.0000
313    Default   452.000000         sugar   452.000000  18.0000  18.0000
154    Default   968.000000           try   968.000000  17.0000  17.0000
382    Default   616.000000        health   616.000000  16.0000  16.0000
74     Default  1257.000000          make  1257.000000  15.0000  15.0000
576    Default  1077.000000          take  1077.000000  14.0000  14.0000
16     Default   981.000000          body   981.000000  13.0000  13.0000
991    Default   323.000000         smell   323.000000  12.0000  12.0000
44     Default   689.000000          feel   689.000000  11.0000  11.0000
215    Default   310.000000          face   310.000000  10.0000  10.0000
89     Default   826.000000       natural   826.000000   9.0000   9.0000
60     Default   680.000000    ingredient   680.000000   8.0000   8.0000
456    Default   854.000000         great   854.000000   7.0000   7.0000
481    Default   417.000000    supplement   417.000000   6.0000   6.0000
160    Default   288.000000          wash   288.000000   5.0000   5.0000
1896   Default   113.000000          pain   113.000000   4.0000   4.0000
316    Default   385.000000       support   385.000000   3.0000   3.0000
932    Default   147.000000         sleep   147.000000   2.0000   2.0000
279    Default   288.000000        powder   288.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
25      Topic7    16.927910   conditioner    17.843544   4.3891  -4.6150
2059    Topic7    11.629882        volume    12.546518   4.3659  -4.9904
151     Topic7     8.904828         towel     9.821396   4.3438  -5.2574
11851   Topic7     8.078850     detangler     9.000446   4.3338  -5.3547
3164    Topic7     8.062039           pot     8.990501   4.3328  -5.3568
5910    Topic7     7.239171          damp     8.159527   4.3221  -5.4645
125     Topic7     7.011462         scalp     7.927789   4.3190  -5.4964
2003    Topic7     6.933552         messy     7.860487   4.3163  -5.5076
2052    Topic7     6.758799         shiny     7.675404   4.3146  -5.5331
10881   Topic7     6.680920    compliment     7.599927   4.3129  -5.5447
1056    Topic7     5.936479  overwhelming     6.865680   4.2964  -5.6629
6649    Topic7     5.313277          comb     6.239535   4.2811  -5.7738
10342   Topic7     4.871924         curly     5.790070   4.2691  -5.8605
48      Topic7     4.807738         frizz     5.723914   4.2674  -5.8737
10337   Topic7     4.696170        winner     5.617677   4.2626  -5.8972
70      Topic7     4.665998          loc